# LangChain RAG for Payment Risk Investigation

This notebook demonstrates a **payment-safe workflow** using LangChain to orchestrate multi-step reasoning, tool calls, conditional routing, observability, evaluation, and validation.

The focus is on:
- Multi-step reasoning with LangChain chains
- Policy retrieval tools
- Safety guardrails and output validation
- Observability and logging for audit

## Architecture Overview

This workflow has the following components:

1. **Policy Retrieval Tool** - Simulates retrieval of relevant payment and compliance policies
2. **Risk Classification Chain** - Classifies transactions into risk categories
3. **Explanation Generation Chain** - Generates structured explanations for flagged transactions
4. **Sequential Workflow** - Chains steps while keeping each step auditable
5. **Output Validation** - Ensures AI output is safe and does not contain forbidden instructions

**Key principle:** AI assists analysts; it does not take payment actions.

## 1. Install Dependencies

Install LangChain and Ollama packages. Ollama runs locally, no API key needed.

**Prerequisites**: You need Ollama installed and running with a model downloaded.
Run this in your terminal first:
```bash
# Install Ollama from https://ollama.ai
ollama pull llama3.1:8b
```

In [1]:
pip install langchain langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.2/603.2 kB 18.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.tools import BaseTool
from langchain_ollama import ChatOllama
from typing import Type
from pydantic import BaseModel, Field

## 3. Define Policy Retrieval Tool

This tool simulates retrieval of relevant payment and compliance policies. In production, this would interface with a vector database or policy knowledge store.

In [4]:
class PolicyRetrievalInput(BaseModel):
    query: str = Field(description="Query to search for relevant policies")

class PolicyRetrievalTool(BaseTool):
    name: str = "policy_retrieval"
    description: str = "Retrieve payment and compliance policies relevant to a transaction."
    args_schema: Type[BaseModel] = PolicyRetrievalInput

    def _run(self, query: str) -> str:
        policies = {
            "high_amount": "Transactions above $500 require additional verification per Policy 4.2.1",
            "cross_border": "Cross-border payments to high-risk countries require enhanced due diligence per Policy 3.1.5",
            "new_merchant": "First-time merchants require manual review per Policy 2.3.4",
            "default": "Standard payment processing policies apply per Policy 1.0.0"
        }
        
        query_lower = query.lower()
        if "amount" in query_lower or "high" in query_lower:
            return policies["high_amount"]
        elif "cross" in query_lower or "country" in query_lower or "brazil" in query_lower:
            return policies["cross_border"]
        elif "merchant" in query_lower or "new" in query_lower:
            return policies["new_merchant"]
        return policies["default"]

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("Async retrieval not implemented")

policy_tool = PolicyRetrievalTool()
print(f"Tool created: {policy_tool.name}")

Tool created: policy_retrieval


### Test the Policy Retrieval Tool

In [5]:
print("Test 1 - High amount:", policy_tool.run("high amount transaction"))
print("Test 2 - Cross-border:", policy_tool.run("payment to Brazil"))
print("Test 3 - New merchant:", policy_tool.run("new merchant review"))

Test 1 - High amount: Transactions above $500 require additional verification per Policy 4.2.1
Test 2 - Cross-border: Cross-border payments to high-risk countries require enhanced due diligence per Policy 3.1.5
Test 3 - New merchant: First-time merchants require manual review per Policy 2.3.4


## 4. Define Prompt Templates

We separate risk classification from explanation generation to maintain auditability.

In [6]:
risk_classification_prompt = PromptTemplate(
    input_variables=['transaction'],
    template="""
You are a payment risk analyst assistant. Your role is to classify transactions, not to make decisions.

Classify the following transaction into a risk category (High, Medium, Low).
Provide a confidence score between 0 and 1.
Only classify, do not recommend any action.

Transaction details: {transaction}

Respond in this format:
Risk Category: [High/Medium/Low]
Confidence: [0.0-1.0]
Reasoning: [Brief explanation]
"""
)

explanation_prompt = PromptTemplate(
    input_variables=['risk_classification', 'policy_context'],
    template="""
You are a payment risk analyst assistant. Generate a structured explanation for the risk classification.
Use the policy context provided and do not suggest any action.

Risk Classification:
{risk_classification}

Policy Context:
{policy_context}

Generate a JSON response with the following fields:
- category: the risk category
- confidence: the confidence score
- explanation: detailed explanation referencing the policies
- policy_references: list of policy numbers cited
"""
)

print("Prompt templates defined successfully!")

Prompt templates defined successfully!


## 5. Setup LLM

Configure Ollama with Llama 3.1 8B model (runs locally, no API key needed).

In [7]:
llm = ChatOllama(model='llama3.1:8b', temperature=0)
print(f"LLM configured: {llm.model}")

LLM configured: llama3.1:8b


## 6. Setup Individual Chains (LCEL)

Create chains using LangChain Expression Language (LCEL) - the modern approach.

In [8]:
risk_chain = risk_classification_prompt | llm | StrOutputParser()

explanation_chain = explanation_prompt | llm | StrOutputParser()

print("Chains created successfully using LCEL!")

Chains created successfully using LCEL!


## 7. Define Example Transaction

Create a sample transaction for testing the workflow.

In [10]:
transaction_example = {
    'id': 'TX12345',
    'amount': 1000,
    'currency': 'BRL',
    'country': 'Brazil',
    'merchant': 'Online Store',
    'payment_method': 'Credit Card',
    'is_first_transaction': True
}

print("Transaction example:")
for key, value in transaction_example.items():
    print(f"  {key}: {value}")

Transaction example:
  id: TX12345
  amount: 1000
  currency: BRL
  country: Brazil
  merchant: Online Store
  payment_method: Credit Card
  is_first_transaction: True


## 8. Execute Workflow Step by Step

Run the transaction through each step of the workflow for maximum visibility.

In [11]:
print("=" * 50)
print("STEP 1: Risk Classification")
print("=" * 50)

risk_classification = risk_chain.invoke({'transaction': str(transaction_example)})
print("\nRisk Classification Result:")
print(risk_classification)

STEP 1: Risk Classification

Risk Classification Result:
Risk Category: Medium
Confidence: 0.7
Reasoning: The transaction is for a large amount (1000 BRL), which may indicate potential fraud or unusual activity. However, the payment method is a credit card, which has some level of built-in security and tracking capabilities. Additionally, this is the first transaction from this merchant, which could be a legitimate new customer.


In [12]:
print("=" * 50)
print("STEP 2: Policy Retrieval")
print("=" * 50)

policy_context = policy_tool.run(f"transaction {transaction_example['country']} amount {transaction_example['amount']}")
print(f"\nRetrieved Policy: {policy_context}")

STEP 2: Policy Retrieval

Retrieved Policy: Transactions above $500 require additional verification per Policy 4.2.1


In [13]:
print("=" * 50)
print("STEP 3: Explanation Generation")
print("=" * 50)

final_explanation = explanation_chain.invoke({
    'risk_classification': risk_classification,
    'policy_context': policy_context
})
print("\nFinal Explanation:")
print(final_explanation)

STEP 3: Explanation Generation

Final Explanation:
Here is the structured explanation for the risk classification:

```json
{
  "category": "Medium",
  "confidence": 0.7,
  "explanation": "The transaction is classified as medium-risk due to its large amount (1000 BRL), which may indicate potential fraud or unusual activity. However, the payment method used is a credit card, which has built-in security and tracking capabilities. Additionally, this is the first transaction from this merchant, which could be a legitimate new customer.",
  "policy_references": [
    "Policy 4.2.1"
  ]
}
```

Note: The explanation references Policy 4.2.1, which requires additional verification for transactions above $500. However, since the confidence score is 0.7 and the risk category is Medium, no further action is suggested based on this policy alone.


## 9. Observability & Audit Logging

Log essential information for auditing and monitoring.

In [14]:
import json
from datetime import datetime

log_entry = {
    'timestamp': datetime.now().isoformat(),
    'transaction_id': transaction_example['id'],
    'input': transaction_example,
    'risk_classification': risk_classification,
    'policy_retrieved': policy_context,
    'final_explanation': final_explanation,
    'metadata': {
        'policy_tool_used': 'PolicyRetrievalTool',
        'llm_model': 'llama3.1:8b',
        'prompt_versions': {
            'risk_chain': 'v1',
            'explanation_chain': 'v1'
        }
    }
}

print("Audit Log Entry:")
print(json.dumps(log_entry, indent=2, default=str))

Audit Log Entry:
{
  "timestamp": "2025-12-17T14:27:43.574033",
  "transaction_id": "TX12345",
  "input": {
    "id": "TX12345",
    "amount": 1000,
    "currency": "BRL",
    "country": "Brazil",
    "merchant": "Online Store",
    "payment_method": "Credit Card",
    "is_first_transaction": true
  },
  "risk_classification": "Risk Category: Medium\nConfidence: 0.7\nReasoning: The transaction is for a large amount (1000 BRL), which may indicate potential fraud or unusual activity. However, the payment method is a credit card, which has some level of built-in security and tracking capabilities. Additionally, this is the first transaction from this merchant, which could be a legitimate new customer.",
  "policy_retrieved": "Transactions above $500 require additional verification per Policy 4.2.1",
  "final_explanation": "Here is the structured explanation for the risk classification:\n\n```json\n{\n  \"category\": \"Medium\",\n  \"confidence\": 0.7,\n  \"explanation\": \"The transaction

## 10. Output Validation (Safety Guardrail)

Ensure AI output is safe and does not contain forbidden instructions. This is critical for payment systems where AI should assist, not decide.

In [15]:
forbidden_phrases = [
    "approve",
    "reject", 
    "block the payment",
    "cancel the transaction",
    "proceed with payment",
    "authorize"
]

def validate_output(response: str, forbidden: list) -> dict:
    response_lower = response.lower()
    violations = [phrase for phrase in forbidden if phrase in response_lower]
    return {
        'is_valid': len(violations) == 0,
        'violations': violations
    }

validation_result = validate_output(
    final_explanation, 
    forbidden_phrases
)

print(f"Output Validation Result:")
print(f"  Is Valid: {validation_result['is_valid']}")
if validation_result['violations']:
    print(f"  Violations Found: {validation_result['violations']}")
else:
    print("  No forbidden phrases detected - output is safe!")

Output Validation Result:
  Is Valid: True
  No forbidden phrases detected - output is safe!


## 11. Complete Workflow Function (Alternative)

For a more automated approach, wrap the workflow in a function.

In [18]:
def run_full_workflow(transaction: dict) -> dict:
    # Step 1: Risk Classification
    risk_result = risk_chain.invoke({'transaction': str(transaction)})
    
    # Step 2: Policy Retrieval
    policy_context = policy_tool.run(
        f"transaction {transaction.get('country', '')} amount {transaction.get('amount', '')}"
    )
    
    # Step 3: Explanation Generation
    explanation_result = explanation_chain.invoke({
        'risk_classification': risk_result,
        'policy_context': policy_context
    })
    
    return {
        'risk_classification': risk_result,
        'policy_context': policy_context,
        'final_explanation': explanation_result
    }

print("Full workflow function created!")

Full workflow function created!


In [19]:
transaction_2 = {
    'id': 'TX67890',
    'amount': 50,
    'currency': 'USD',
    'country': 'USA',
    'merchant': 'Coffee Shop',
    'payment_method': 'Debit Card'
}

full_result = run_full_workflow(transaction_2)

print("\n" + "=" * 50)
print("FULL WORKFLOW RESULT")
print("=" * 50)
print(f"\nRisk Classification:\n{full_result['risk_classification']}")
print(f"\nPolicy Retrieved:\n{full_result['policy_context']}")
print(f"\nFinal Explanation:\n{full_result['final_explanation']}")


FULL WORKFLOW RESULT

Risk Classification:
Risk Category: Low
Confidence: 0.8
Reasoning: The transaction amount is relatively small ($50), and it's a domestic payment (USA) using a debit card, which suggests a low risk of fraud or chargeback. However, I wouldn't classify it as extremely low-risk due to the possibility of friendly fraud or unauthorized transactions.

Policy Retrieved:
Transactions above $500 require additional verification per Policy 4.2.1

Final Explanation:
Here is the structured explanation for the risk classification in JSON format:

```json
{
  "category": "Low",
  "confidence": 0.8,
  "explanation": "The transaction amount is relatively small ($50), and it's a domestic payment (USA) using a debit card, which suggests a low risk of fraud or chargeback. However, I wouldn't classify it as extremely low-risk due to the possibility of friendly fraud or unauthorized transactions.",
  "policy_references": [
    {
      "policy_number": "4.2.1",
      "description": "Tra

## Summary

This notebook demonstrated:

1. **Local LLM Setup** - Using Ollama with Llama 3.1 (no API key needed)
2. **Policy Retrieval Tool** - Simulated retrieval of internal policies
3. **Prompt Templates** - Separated risk classification from explanation generation
4. **LLM Setup** - Deterministic model behavior with temperature=0
5. **Chain Setup** - Individual LLM steps with output keys for auditability
6. **Step-by-Step Execution** - Full visibility into each workflow step
7. **Observability** - Comprehensive logging for audit and monitoring
8. **Output Validation** - Safety guardrails to prevent forbidden actions
9. **Sequential Chain** - Automated workflow option

### Key Principles

- **AI assists analysts; it does not take payment actions**
- **Every step is auditable and logged**
- **Output validation ensures safety**
- **Policy context grounds the AI responses**

### Extensions

This can be extended with:
- RouterChain for conditional routing based on risk level
- Human-in-the-loop review for high-risk transactions
- Integration with real vector databases for policy retrieval
- LangSmith for production observability